In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score

import numpy as np



In [2]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

for split_label in range(1, 4):
    predictions = pd.read_csv(f"predictions_split{split_label}.csv")
    y_test = pd.read_csv(f"y_test_split{split_label}.csv")
    
    # Compute AUC for each task
    for column in y_test.columns:
        # Get the predictions and true values for the current task
        y_true = y_test[column].values
        y_pred = predictions[column].values

        # Filter out -1 values
        valid_indices = y_true != -1
        y_true_valid = y_true[valid_indices]
        y_pred_valid = y_pred[valid_indices]

        # Compute AUC if there are valid data points
        if len(y_true_valid) > 0 and len(np.unique(y_true_valid)) > 1:
            auc = roc_auc_score(y_true_valid, y_pred_valid)
        else:
            auc = np.nan  # Not enough data to compute AUC

        all_auc_scores.append({"Fold": split_label, "Task": column, "Cel_counter_model": auc})

# Convert to DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

auc_df

,Fold,Task,Cel_counter_model
0,1,1,0.454545
1,1,2,0.685185
2,1,3,0.541667
3,1,4,0.571429
4,1,5,0.777778
...,...,...,...
622,3,205,0.484848
623,3,206,0.516667
624,3,207,0.200000
625,3,208,0.115385


In [3]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,Fold,Cel_counter_model
0,1,2.0,0.648375
1,10,2.0,0.530050
2,100,2.0,0.457325
3,101,2.0,0.343655
4,102,2.0,0.424992
...,...,...,...
204,95,2.0,0.470187
205,96,2.0,0.790890
206,97,2.0,0.760582
207,98,2.0,0.517021


In [4]:
len(mean_auc[mean_auc["Cel_counter_model"]==1])

0

In [5]:
len(mean_auc[mean_auc["Cel_counter_model"]>0.90])

0

In [10]:
mean_auc.to_csv("mean_auc_scores.csv")

In [11]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("meadian_auc_scores.csv")
median_auc

,Task,Fold,Cel_counter_model
0,1,2.0,0.673913
1,10,2.0,0.536842
2,100,2.0,0.510417
3,101,2.0,0.314516
4,102,2.0,0.422222
...,...,...,...
204,95,2.0,0.470187
205,96,2.0,0.818182
206,97,2.0,0.750000
207,98,2.0,0.473118


In [12]:
len(median_auc[median_auc["Cel_counter_model"]==1])

0

In [13]:
len(median_auc[median_auc["Cel_counter_model"]>0.90])

0

In [14]:
import numpy as np
mean_auc["Task"] = mean_auc["Task"].astype(np.int64)
mean_auc

,Task,Fold,Cel_counter_model
0,1,2.0,0.648375
1,10,2.0,0.530050
2,100,2.0,0.457325
3,101,2.0,0.343655
4,102,2.0,0.424992
...,...,...,...
204,95,2.0,0.470187
205,96,2.0,0.790890
206,97,2.0,0.760582
207,98,2.0,0.517021


In [15]:
assays = pd.read_csv("data/column-assay-index.csv", sep = ",").reset_index()
assays["index"] = assays["index"]+1
assays = assays[["CHEMBL_ID", "index"]]
assays

,CHEMBL_ID,index
0,CHEMBL1040691,1
1,CHEMBL1614544,2
2,CHEMBL1613914,3
3,CHEMBL1613836,4
4,CHEMBL1613838,5
...,...,...
204,CHEMBL3214913,205
205,CHEMBL3214967,206
206,CHEMBL3214906,207
207,CHEMBL3214970,208


In [16]:
df = pd.merge(mean_auc, assays, left_on="Task", right_on="index").sort_values("index")
df

,Task,Fold,Cel_counter_model,CHEMBL_ID,index
0,1,2.0,0.648375,CHEMBL1040691,1
111,2,2.0,0.598917,CHEMBL1614544,2
132,3,2.0,0.339766,CHEMBL1613914,3
143,4,2.0,0.474883,CHEMBL1613836,4
154,5,2.0,0.634151,CHEMBL1613838,5
...,...,...,...,...,...
118,205,2.0,0.545692,CHEMBL3214913,205
119,206,2.0,0.472080,CHEMBL3214967,206
120,207,2.0,0.288889,CHEMBL3214906,207
121,208,2.0,0.493480,CHEMBL3214970,208


In [17]:
df = df[["CHEMBL_ID", "Cel_counter_model"]]
df.to_csv("mean_auc_scores.csv", index=False)
df

,CHEMBL_ID,Cel_counter_model
0,CHEMBL1040691,0.648375
111,CHEMBL1614544,0.598917
132,CHEMBL1613914,0.339766
143,CHEMBL1613836,0.474883
154,CHEMBL1613838,0.634151
...,...,...
118,CHEMBL3214913,0.545692
119,CHEMBL3214967,0.472080
120,CHEMBL3214906,0.288889
121,CHEMBL3214970,0.493480
